In [1]:
import locale
def gpe(x=None):
    return "UTF-8"
locale.getpreferredencoding = gpe

modelpath = r'F:\nllb-gos\hfacemodels'
modelname = 'facebook/nllb-200-distilled-1.3B' # 3.3B might be too big for my puny 3060 12GB (1.3B was too big for 1660ti) # note: there's distilled and non distilled models!
MODEL_SAVE_PATH = '../NLLB-GOS/models/nllb-multi-gos-1.3B-v0'

## 1. Exploring the data

In this section, I try to understand what is the training data that I have, and how suitable it is for fine-tuning a NLLB model.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

# Define language codes
source_langs = ["nld", "eng", "deu", "spa", "afr"]
target_lang = "gos"
target_lang_long = target_lang+"_Latn"

class ParallelCorpus:
    def __init__(self, source_lang, target_lang):
        self.source_lang = source_lang
        self.target_lang = target_lang
        self.source_lang_long = source_lang+'_Latn'
        self.target_lang_long = target_lang+'_Latn'
        
        # Load the parallel sentences for the given language pair
        self.df = self.load_and_format_parallel_sentences()
        
        # Split the data into train, validation, and test sets
        self.df_train, self.df_temp = train_test_split(self.df, test_size=0.02, random_state=9358)
        self.df_validate, self.df_test = train_test_split(self.df_temp, test_size=0.5, random_state=9358)
        
        # Clean up temporary DataFrame
        del self.df_temp

    def load_and_format_parallel_sentences(self):
        # Load sentences for the source and target language
        source_sentences = pd.read_csv(
            f"../tatoeba/{self.source_lang}_sentences.tsv.bz2", 
            sep="\t", header=None, names=["id", "language", "source_sentence"]
        )
        target_sentences = pd.read_csv(
            f"../tatoeba/{self.target_lang}_sentences.tsv.bz2", 
            sep="\t", header=None, names=["id", "language", "target_sentence"]
        )
        
        # Load links
        link_sentences = pd.read_csv(
            "../tatoeba/links.tar.bz2", 
            sep="\t", header=None, names=["origin", "translation"]
        )
        
        # Merge datasets to create parallel sentence pairs
        df_parallel = link_sentences.merge(source_sentences, left_on="origin", right_on="id")\
                                    .merge(target_sentences, left_on="translation", right_on="id")\
                                    [["target_sentence", "source_sentence"]]
        
        # Add source language code
        df_parallel["source_lang_code"] = self.source_lang
        
        return df_parallel

# Create a list of ParallelCorpus instances for each source language
corpus_objects = [ParallelCorpus(lang, target_lang) for lang in tqdm(source_langs)]

# Example: Access the training, validation, and test DataFrames for the first source language in the list
corpus_objects[0].df_test.sample(4)


In [3]:
# import requests

# url = "https://api.tatoeba.org/unstable/sentences"
# params = {
#     "lang": "gos",
#     "q": "noam", #search in tatoeba database here
#     "include_unapproved": "yes"
# }

# response = requests.get(url, params=params)

# # Check if the request was successful
# if response.status_code == 200:
#     data = response.json()
#     print('success, found',len(data['data']),'\n')
#     #print(data)  # Print the response JSON
# else:
#     print(f"Request failed with code: {response.status_code}")

# for i in data['data']:
#     print(i['text'])

# 4. Adding a new language tag to the tokenizer and model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer
from gc import collect
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# we need transformers<=4.33 for this to work.

def fix_tokenizer(tokenizer, new_lang='gos_Latn'):
    """
    Add a new language token to the tokenizer vocabulary
    (this should be done each time after its initialization)
    """
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    # always move "mask" to the last position
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    # clear the added token encoder; otherwise a new token may end up there by mistake
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

def cleanup():
    """Try to free GPU memory"""
    collect()
    torch.cuda.empty_cache()

tokenizer = NllbTokenizer.from_pretrained(modelname, cache_dir=modelpath)
print(tokenizer.convert_ids_to_tokens([256202, 256203]))

fix_tokenizer(tokenizer)
print(tokenizer.convert_ids_to_tokens([256202, 256203, 256204])) # ['zul_Latn', 'tyv_Cyrl', '<mask>']
print(tokenizer.convert_tokens_to_ids(['zul_Latn', 'gos_Latn', '<mask>'])) # [256202, 256203, 256204]

In [ ]:
cleanup()
model = AutoModelForSeq2SeqLM.from_pretrained(modelname, cache_dir=modelpath, device_map={"": "cuda:0"})
# for name, module in model.named_modules(): print(name)

In [ ]:
added_token_id = tokenizer.convert_tokens_to_ids(target_lang_long)
#similar_lang_id = tokenizer.convert_tokens_to_ids('fry_Latn')
#similar_lang_id = tokenizer.convert_tokens_to_ids('nds_Latn')
similar_lang_id = tokenizer.convert_tokens_to_ids('nld_Latn')
print(added_token_id, similar_lang_id)

model.resize_token_embeddings(len(tokenizer))

# moving the embedding for "mask" to its new position
model.model.shared.weight.data[added_token_id+1] = model.model.shared.weight.data[added_token_id]
# initializing new language token with a token of a similar language
model.model.shared.weight.data[added_token_id] = model.model.shared.weight.data[similar_lang_id]

model.resize_token_embeddings(len(tokenizer))

# 5. Preparing the training loop

In [ ]:
import random
import numpy as np
from tqdm.auto import  trange
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup
import sys
import re
import typing as tp
import unicodedata
from sacremoses import MosesPunctNormalizer

model.cuda();

optimizer = Adafactor(
    [p for p in model.parameters() if p.requires_grad],
    scale_parameter=False,
    relative_step=False,
    lr=1e-4,
    clip_threshold=1.0,
    weight_decay=1e-3,
)

batch_size = 12 # 11 is fine, 12 maybe??
max_length = 125
warmup_steps = 100
training_steps = int(800*8/batch_size)

losses = []
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)

mpn = MosesPunctNormalizer(lang="en")
mpn.substitutions = [
    (re.compile(r), sub) for r, sub in mpn.substitutions
]

def get_non_printing_char_replacer(replace_by: str = " ") -> tp.Callable[[str], str]:
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        # same as \p{C} in perl
        # see https://www.unicode.org/reports/tr44/#General_Category_Values
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(line) -> str:
        return line.translate(non_printable_map)

    return replace_non_printing_char

replace_nonprint = get_non_printing_char_replacer(" ")

def preproc(text):
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    # replace 𝓕𝔯𝔞𝔫𝔠𝔢𝔰𝔠𝔞 by Francesca
    clean = unicodedata.normalize("NFKC", clean)
    return clean

def get_batch_pairs(batch_size, corpus_objects=corpus_objects):
    corpus = random.choice(corpus_objects)
    batch = corpus.df_train.sample(n=batch_size)

    batch['source_sentence'] = batch['source_sentence'].apply(preproc)
    batch['target_sentence'] = batch['target_sentence'].apply(preproc)
    
    source_lang_long = corpus.source_lang_long
    target_lang_long = corpus.target_lang_long
    
    xx = batch['source_sentence'].tolist()
    yy = batch['target_sentence'].tolist()
    
    if random.getrandbits(1):
        return xx, yy, source_lang_long, target_lang_long
    else:
        return yy, xx, target_lang_long, source_lang_long

display(get_batch_pairs(2))

## 6. The training loop

In [ ]:
model.train()
x, y, loss = None, None, None
cleanup()

tq = trange(len(losses), training_steps)
for i in tq:
    xx, yy, lang1, lang2 = get_batch_pairs(batch_size)
    try:
        tokenizer.src_lang = lang1
        x = tokenizer(xx, return_tensors='pt', padding='longest', truncation=True, max_length=max_length).to(model.device)
        tokenizer.src_lang = lang2
        y = tokenizer(yy, return_tensors='pt', padding='longest', truncation=True, max_length=max_length).to(model.device)
        y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100

        loss = model(**x, labels=y.input_ids).loss
        loss.backward()
        losses.append(loss.item())

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        scheduler.step()

    except RuntimeError as e:
        optimizer.zero_grad(set_to_none=True)
        x, y, loss = None, None, None
        cleanup()
        print('error', max(len(s) for s in xx + yy), e)
        continue

    if i % 100 == 0 and i > 0:
        print(i, "mean loss of last 100 steps:\t", np.mean(losses[-100:]))
        if i > 3*training_steps/4:
            model.save_pretrained(MODEL_SAVE_PATH)
            tokenizer.save_pretrained(MODEL_SAVE_PATH)

In [ ]:
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

In [ ]:
pd.Series(losses).ewm(100).mean().plot();

In [14]:
def translate(text, src_lang='nld_Latn', tgt_lang='gos_Latn', a=16, b=1.5, max_input_length=700, **kwargs):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_input_length)
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        **kwargs
    )
    #print(inputs.input_ids.shape[1], result.shape[1])
    return tokenizer.batch_decode(result, skip_special_tokens=True)

In [ ]:
s = 'Llegaste a tiempo?'
langA = 'spa_Latn'
langB = 'gos_Latn'

print(s)
t = translate(s, langA, langB)
print(t)
t_back = translate(t, langB, langA)
print(t_back)

In [ ]:
xx, yy, lang1, lang2 = get_batch_pairs(1)
print(xx)
print(yy)
model.eval()
print(translate(xx[0], lang1, lang2, no_repeat_ngram_size=3, num_beams=5))

In [ ]:
1/0

## 7. Using the model

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import NllbTokenizer, AutoModelForSeq2SeqLM, AutoConfig
from tqdm.auto import tqdm, trange

In [29]:
# df_parallel = pd.read_csv('/gd/MyDrive/datasets/nlp/tyvan/for_translator.csv')
# df_parallel.dropna(subset=['ru', 'tyv'], inplace=True)
# df_train, df_devtest = train_test_split(df_parallel, test_size=1000, random_state=1)
# df_dev, df_test = train_test_split(df_devtest, test_size=0.5, random_state=1)

In [30]:
# this code is adapted from  the Stopes repo of the NLLB team
# https://github.com/facebookresearch/stopes/blob/main/stopes/pipelines/monolingual/monolingual_line_processor.py#L214

import re
import sys
import typing as tp
import unicodedata
from sacremoses import MosesPunctNormalizer


mpn = MosesPunctNormalizer(lang="en")
mpn.substitutions = [
    (re.compile(r), sub) for r, sub in mpn.substitutions
]


def get_non_printing_char_replacer(replace_by: str = " ") -> tp.Callable[[str], str]:
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        # same as \p{C} in perl
        # see https://www.unicode.org/reports/tr44/#General_Category_Values
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(line) -> str:
        return line.translate(non_printable_map)

    return replace_non_printing_char

replace_nonprint = get_non_printing_char_replacer(" ")

def preproc(text):
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    # replace 𝓕𝔯𝔞𝔫𝔠𝔢𝔰𝔠𝔞 by Francesca
    clean = unicodedata.normalize("NFKC", clean)
    return clean

In [ ]:
model_load_name = '../NLLB-GOS/models/nllb-nld-gos-v6'
model = AutoModelForSeq2SeqLM.from_pretrained(model_load_name).cuda()
tokenizer = NllbTokenizer.from_pretrained(model_load_name)
# fix_tokenizer(tokenizer)

In [32]:
def fix_tokenizer(tokenizer, new_lang='gos_Latn'):
    """ Add a new language token to the tokenizer vocabulary (this should be done each time after its initialization) """
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    # always move "mask" to the last position
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    # clear the added token encoder; otherwise a new token may end up there by mistake
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

fix_tokenizer(tokenizer)

In [33]:
def translate(text, src_lang='nld_Latn', tgt_lang='eng_Latn', a=32, b=3, max_input_length=1024, num_beams=4, **kwargs):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_input_length)
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams,
        **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)

In [ ]:
t = 'De vertalingen zijn niet perfect, maar er gaat al best veel goed.' #"мөңгүн үр чыткаш карарар"
print(translate(t, 'nld_Latn', 'gos_Latn'))
#t_translated translate(t, 'nld_Latn', 'gos_Latn')
# ['серебро от времени чернеет']

In [ ]:
translate(t, 'nld_Latn', 'gos_Latn', do_sample=True, num_beams=2, temperature=1.5)

In [ ]:
df_validate['nld_translated'] = [translate(t, 'gos_Latn', 'nld_Latn')[0] for t in tqdm(df_validate.gos)]
df_validate['gos_translated'] = [translate(t, 'nld_Latn', 'gos_Latn')[0] for t in tqdm(df_validate.nld)]

In [ ]:
df_validate.sample(15)

In [38]:
import sacrebleu
bleu_calc = sacrebleu.BLEU()
chrf_calc = sacrebleu.CHRF(word_order=2)  # this metric is called ChrF++

In [ ]:
!pip install editdistance

In [ ]:
import editdistance

def ed_similarity(text1, text2):
    return max(0, 1 - editdistance.eval(text1, text2) / min(len(text1), len(text2)))

print(ed_similarity('кот', 'собака'))
print(ed_similarity('кот', 'кит'))

In [ ]:
display(pd.Series([ed_similarity(row.nld, row.nld_translated) for row in df_validate.itertuples()]).describe())
display(pd.Series([ed_similarity(row.gos, row.gos_translated) for row in df_validate.itertuples()]).describe())

In [ ]:
print(bleu_calc.corpus_score(df_validate['rus_translated2'].tolist(), [df_validate['ru'].tolist()]))
print(chrf_calc.corpus_score(df_validate['rus_translated2'].tolist(), [df_validate['ru'].tolist()]))
print(bleu_calc.corpus_score(df_validate['tyv_translated2'].tolist(), [df_validate['tyv'].tolist()]))
print(chrf_calc.corpus_score(df_validate['tyv_translated2'].tolist(), [df_validate['tyv'].tolist()]))

In [ ]:
df_validate['rus2eng'] = [translate(t, 'tyv_Cyrl', 'eng_Latn')[0] for t in tqdm(df_validate.tyv)]
df_validate['tyv2eng'] = [translate(t, 'rus_Cyrl', 'eng_Latn')[0] for t in tqdm(df_validate.ru)]

Results with num_beams=1:
```
V1
BLEU = 23.21 51.2/29.1/18.0/11.8 (BP = 0.978 ratio = 0.978 hyp_len = 2273 ref_len = 2324)
chrF2++ = 47.88
BLEU = 22.03 51.5/29.7/17.9/10.4 (BP = 0.952 ratio = 0.953 hyp_len = 2260 ref_len = 2371)
chrF2++ = 49.37
V2
BLEU = 24.08 50.9/29.5/19.1/12.3 (BP = 0.988 ratio = 0.988 hyp_len = 2297 ref_len = 2324)
chrF2++ = 48.96
BLEU = 22.50 50.5/28.5/17.7/11.1 (BP = 0.974 ratio = 0.974 hyp_len = 2310 ref_len = 2371)
chrF2++ = 48.85
V3
BLEU = 22.25 49.8/27.8/17.2/11.0 (BP = 0.983 ratio = 0.983 hyp_len = 2284 ref_len = 2324)
chrF2++ = 47.89
BLEU = 25.28 52.2/31.2/20.0/13.1 (BP = 0.989 ratio = 0.989 hyp_len = 2346 ref_len = 2371)
chrF2++ = 51.87
````

Results with 4 beams:
```
V1
BLEU = 24.14 52.5/30.4/18.9/12.1 (BP = 0.981 ratio = 0.981 hyp_len = 2281 ref_len = 2324)
chrF2++ = 49.49
BLEU = 23.41 52.1/31.0/18.9/11.3 (BP = 0.966 ratio = 0.967 hyp_len = 2292 ref_len = 2371)
chrF2++ = 50.89
V2
BLEU = 25.18 52.4/31.3/20.4/13.3 (BP = 0.976 ratio = 0.976 hyp_len = 2269 ref_len = 2324)
chrF2++ = 49.85
BLEU = 23.22 51.6/29.4/18.3/11.6 (BP = 0.975 ratio = 0.975 hyp_len = 2312 ref_len = 2371)
chrF2++ = 49.87
V3
BLEU = 23.06 51.1/29.1/18.1/11.5 (BP = 0.978 ratio = 0.978 hyp_len = 2273 ref_len = 2324)
chrF2++ = 48.56
BLEU = 26.12 53.4/32.5/21.0/13.6 (BP = 0.985 ratio = 0.985 hyp_len = 2336 ref_len = 2371)
chrF2++ = 52.60
```

Which means:
* For all directions and models, beam search improves the results
* Longer training builds up quality for Tyvan, but decreases it for Russian.

```
                                  | tyv->rus | rus->tyv
Model v1 (no vocabulary update):  |
    no beam search                |   23.21  |  22.03
    num_beams = 4                 |   24.14  |  23.41
Model v2 (with vocabulary update):|
    no beam search                |   24.08  |  22.50
    num_beams = 4                 |   25.18  |  23.22
```

## Publishing the model to HF

In [ ]:
!huggingface-cli login

In [ ]:
from transformers import NllbTokenizer, AutoModelForSeq2SeqLM, AutoConfig

In [ ]:
def fix_tokenizer(tokenizer, new_lang='tyv_Cyrl'):
    """ Add a new language token to the tokenizer vocabulary (this should be done each time after its initialization) """
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    # always move "mask" to the last position
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    # clear the added token encoder; otherwise a new token may end up there by mistake
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

In [ ]:
model_load_name = '/gd/MyDrive/models/nllb-rus-tyv-v1'
model = AutoModelForSeq2SeqLM.from_pretrained(model_load_name)
tokenizer = NllbTokenizer.from_pretrained(model_load_name)
fix_tokenizer(tokenizer)

In [ ]:
upload_repo = "slone/nllb-rus-tyv-v1"
tokenizer.push_to_hub(upload_repo)
model.push_to_hub(upload_repo)

In [ ]:
model_load_name = '/gd/MyDrive/models/nllb-rus-tyv-v2-extvoc'
tokenizer = NllbTokenizer.from_pretrained(model_load_name)
cfg = AutoConfig.from_pretrained(model_load_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_load_name + "/pytorch_model_60k.bin", config=cfg)

In [ ]:
upload_repo = "slone/nllb-rus-tyv-v2-extvoc"
tokenizer.push_to_hub(upload_repo)
model.push_to_hub(upload_repo)

In [ ]:
print(tokenizer.convert_ids_to_tokens([256202, 256203, 256204])) # ['zul_Latn', 'tyv_Cyrl', '<mask>']
print(tokenizer.convert_tokens_to_ids(['zul_Latn', 'tyv_Cyrl', '<mask>'])) # [256202, 256203, 256204]
# this is consistent now, wow!

In [ ]:
MODEL_URL = 'slone/nllb-rus-tyv-v1'
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_URL)
tokenizer = NllbTokenizer.from_pretrained(MODEL_URL, force_download=True)
fix_tokenizer(tokenizer)

In [ ]:
def translate(
    text,
    model,
    tokenizer,
    src_lang='rus_Cyrl',
    tgt_lang='tyv_Cyrl',
    max_length='auto',
    num_beams=4,
    no_repeat_ngram_size=4,
    n_out=None,
    **kwargs
):
    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    if max_length == 'auto':
        max_length = int(32 + 2.0 * encoded.input_ids.shape[1])
    model.eval()
    generated_tokens = model.generate(
        **encoded.to(model.device),
        forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang],
        max_length=max_length,
        num_beams=num_beams,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_return_sequences=n_out or 1,
        **kwargs
    )
    out = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    if isinstance(text, str) and n_out is None:
        return out[0]
    return out